In [1]:
# !pip install mysql-connector-python
# !pip install pyspark

In [2]:
import mysql.connector
from pyspark.sql import SparkSession
import csv
import random
from pyspark.sql.functions import lit, col, when

In [3]:
# mydb = mysql.connector.connect(
#   host="localhost",
#   user="root",
#   password="",
#   database="spark_train_one"
# )


In [4]:
class ToDictMixin:
    def to_dict(self):
        return {k: v for k, v in vars(self).items() if not k.startswith("__")}



In [5]:
class Armor(ToDictMixin):
    
    id_counter = 1
    
    def __init__(self, name: str, armor: int = 1, armor_magik: int = 0, mana: int = 0,
                 durability: int = 5, max_durability: int = None):
        self.id = Armor.id_counter
        Armor.id_counter += 1
        self.name = name
        self.armor = armor
        self.armor_magik = armor_magik
        self.mana = mana
        self.durability = durability
        self.max_durability = max_durability if max_durability is not None else durability
        self.broken = False

In [6]:
class Weapon(ToDictMixin):
    
    id_counter = 1
    
    def __init__(self, name: str, atk: int = 1, atk_magik: int = 0, mana: int = 0,
                 mana_cost: int = 1, durability: int = 5, max_durability: int = None):
        self.id = Weapon.id_counter
        Weapon.id_counter += 1
        self.name = name
        self.atk = atk
        self.atk_magik = atk_magik
        self.mana = mana
        self.mana_cost = mana_cost
        self.durability = durability
        self.max_durability = max_durability if max_durability is not None else durability
        self.broken = False

In [7]:
class Hero(ToDictMixin):
    
    id_counter = 1
    
    
    
    def __init__(self, name: str, pv: int, max_pv: int = None, atk: int = None, atk_magik: int = None, 
                 mana: int = None, max_mana: int = None, initiative: int = None, experience: int = None, 
                 experience_to_level_up: int = None, level: int = None, speed: int = None, origine_speed: int = None):
        self.id = Hero.id_counter
        Hero.id_counter += 1
        self.name = name
        self.pv = pv
        self.max_pv = max_pv if max_pv is not None else pv
        self.atk = atk if atk is not None else 0
        self.atk_magik = atk_magik if atk_magik is not None else 0
        self.mana = mana if mana is not None else 0
        self.max_mana = max_mana if max_mana is not None else mana
        self.initiative = initiative if initiative is not None else 0
        self.experience = experience if experience is not None else 0
        self.experience_to_level_up = experience_to_level_up if experience_to_level_up is not None else 100
        self.level = level if level is not None else 1
        self.speed = speed if speed is not None else 0
        self.origine_speed = origine_speed if origine_speed is not None else 0
        self.items = []
        self.weapon : Weapon | None = None
        self.armor : Armor | None = None
    
    
    def equip_weapon(self, weapon):
        if isinstance(weapon, Weapon) :
            self.weapon = weapon.id
            return True
        return False
        
    def equip_armor(self, armor):
        if isinstance(armor, Armor) :
            self.armor = armor.id
            return True
        return False
    
    def add_item(self, item):
        if isinstance(armor, Item) :
            self.items.append(item.id) 
            return True
        return False

In [8]:
noms_personnages = [
    "Aria", "Balthazar", "Cassandra", "Darian", "Elena", "Felix", "Gabrielle", "Hadrien",
    "Isadora", "Jasper", "Kaïra", "Landon", "Maïlys", "Nolan", "Olivia", "Philemon",
    "Quintus", "Raphael", "Saoirse", "Thaddeus", "Ursula", "Valentin", "Wilhelm", "Xander",
    "Yara", "Zander", "Aidan", "Brianna", "Caleb", "Dahlia", "Evelyn", "Finnegan", "Gwendolyn",
    "Harvey", "Imogen", "Jaden", "Kendra", "Liam", "Matilda", "Niamh", "Ophelia", "Percival",
    "Qiana", "Riley", "Sebastian", "Tristan", "Uriel", "Violet", "Wyatt", "Xiomara", "Yvette", "Zelda"
]
noms_armors = [
    "Armure de cuir", "Armure de fer", "Armure de plaque", "Armure de mailles", "Armure de glace",
    "Armure d'os", "Armure d'écailles", "Armure de dragon", "Armure de lumière", "Armure de ténèbres",
    "Armure de cristal", "Armure de mithril", "Armure de diamant", "Armure de feu", "Armure de foudre",
    "Armure de glace", "Armure de vent", "Armure de l'eau", "Armure de terre", "Armure de roche",
    "Armure de bois", "Armure de métal", "Armure de magma", "Armure de néant", "Armure de luminescence",
    "Armure de brume", "Armure de tempête", "Armure de mort", "Armure de vie", "Armure de guérison",
    "Armure de force", "Armure de sagesse", "Armure de chance", "Armure de la nature", "Armure des anges",
    "Armure des démons", "Armure des héros", "Armure des rois", "Armure des elfes", "Armure des nains",
    "Armure des gnomes", "Armure des orcs", "Armure des gobelins", "Armure des trolls", "Armure des géants",
    "Armure de lave", "Armure de lumière divine", "Armure de l'ombre", "Armure de l'océan", "Armure des vents",
    "Armure des montagnes"
]
noms_weapons = [
    "Épée de cristal", "Hache de guerre", "Bâton de feu", "Dague empoisonnée", "Arc long",
    "Masse d'armes", "Lance d'argent", "Baguette magique", "Gourdin", "Katana",
    "Fouet de combat", "Fleuret du duelliste", "Marteau de guerre", "Glaive", "Rapière",
    "Arbalète", "Hallebarde", "Gantelet de force", "Canne de mage", "Livre de sortilèges",
    "Fléau", "Arc court", "Cimeterre", "Couteau de lancer", "Poignard",
    "Épée bâtarde", "Hache double", "Bâton de glace", "Dague runique", "Arc de chasse",
    "Marteau de Thor", "Lance de la foudre", "Baguette d'illusion", "Gourdin épineux", "Wakizashi",
    "Fouet barbelé", "Épée courte", "Marteau-piqueur", "Glaive du serpent", "Rapière en argent",
    "Arbalète à répétition", "Hallebarde du jugement", "Gantelet du titan", "Canne à feu", "Livre de connaissances",
    "Fléau des ténèbres", "Arc de la lune noire", "Cimeterre démoniaque", "Couteau de jet empoisonné", "Poignard de l'ombre",
]

In [9]:
# Génération de 50 personnages, armors et weapons aléatoires
hero_list = []
armor_list = []
weapon_list = []
for i in range(50):
    hero = Hero(name=noms_personnages[i], pv=random.randint(50, 200), atk=random.randint(0, 50), atk_magik=random.randint(0, 50), 
                mana=random.randint(0, 100), initiative=random.randint(0, 100), experience=random.randint(0, 1000), 
                experience_to_level_up=random.randint(100, 1000), level=random.randint(1, 10), speed=random.randint(0, 100), 
                origine_speed=random.randint(0, 100))
    hero_list.append(hero)
    
    armor = Armor(name=noms_armors[i], armor=random.randint(1, 10), armor_magik=random.randint(0, 5),
                  mana=random.randint(0, 5), durability=random.randint(1, 10),
                  max_durability=random.randint(10, 20))
    armor_list.append(armor)

    weapon = Weapon(name=noms_weapons[i], atk=random.randint(1, 10), atk_magik=random.randint(0, 5),
                    mana=random.randint(0, 5), mana_cost=random.randint(1, 5),
                    durability=random.randint(1, 10), max_durability=random.randint(10, 20))
    weapon_list.append(weapon)
    
hero_attribute_names = list(hero_list[0].to_dict().keys())
armor_attribute_names = list(armor_list[0].to_dict().keys())
weapon_attribute_names = list(weapon_list[0].to_dict().keys())
hero_list[25].equip_weapon(weapon_list[25])
print(hero_attribute_names)
print(armor_attribute_names)
print(weapon_attribute_names)

['id', 'name', 'pv', 'max_pv', 'atk', 'atk_magik', 'mana', 'max_mana', 'initiative', 'experience', 'experience_to_level_up', 'level', 'speed', 'origine_speed', 'items', 'weapon', 'armor']
['id', 'name', 'armor', 'armor_magik', 'mana', 'durability', 'max_durability', 'broken']
['id', 'name', 'atk', 'atk_magik', 'mana', 'mana_cost', 'durability', 'max_durability', 'broken']


In [10]:
# Écriture de la table dans un fichier CSV
def csv_geenerator(path, items_list):
    with open(path, mode="w", newline='') as file:
        fieldnames = list(items_list[0].to_dict().keys())
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for item in items_list:
            writer.writerow({k: v.encode('utf-8').decode('ascii', 'ignore') if isinstance(v, str) else v for k, v in item.to_dict().items()})



In [11]:
csv_geenerator("assets/csv/hero.csv", hero_list)
csv_geenerator("assets/csv/armor.csv", armor_list)
csv_geenerator("assets/csv/weapon.csv", weapon_list)

In [12]:
# mycursor = mydb.cursor()

# mycursor.execute("SELECT * FROM yourtable")

# myresult = mycursor.fetchall()

# for x in myresult:
#   print(x)


spark = SparkSession.builder.appName('Create_Hero').getOrCreate()

df_hero = spark.read.csv("assets/csv/hero.csv", header=True, inferSchema=True)
df_armor = spark.read.csv("assets/csv/armor.csv", header=True, inferSchema=True)
df_weapon = spark.read.csv("assets/csv/weapon.csv", header=True, inferSchema=True)



In [13]:
# Affichage des données du DataFrame
df_hero.show()
df_armor.show()
df_weapon.show()
# Création d'une vue temporaire à partir du DataFrame
df_hero.createOrReplaceTempView("hero")
df_armor.createOrReplaceTempView("armor")
df_weapon.createOrReplaceTempView("weapon")


+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
| id|     name| pv|max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|
+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
|  1|     Aria|195|   195| 13|       14|   5|       5|        54|       582|                   103|    9|   48|           96|   []|  null| null|
|  2|Balthazar|112|   112| 40|        6|  63|      63|        28|       237|                   587|    2|   83|           60|   []|  null| null|
|  3|Cassandra|160|   160| 17|        7|  60|      60|        96|        23|                   881|    7|   18|           53|   []|  null| null|
|  4|   Darian|132|   132| 15|       46|  13|      13|        87|       432|                   444|    2|   23|           17|   []

In [14]:
print(spark.catalog.listTables())

[Table(name='armor', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True), Table(name='hero', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True), Table(name='weapon', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]


In [15]:

# Exécution d'une requête SQL pour sélectionner les héros avec plus de 100 points de vie
result = spark.sql("SELECT * FROM hero WHERE pv > 100")

# Affichage des résultats
result.show(n=500, truncate=False)

+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
|id |name     |pv |max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|
+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
|1  |Aria     |195|195   |13 |14       |5   |5       |54        |582       |103                   |9    |48   |96           |[]   |null  |null |
|2  |Balthazar|112|112   |40 |6        |63  |63      |28        |237       |587                   |2    |83   |60           |[]   |null  |null |
|3  |Cassandra|160|160   |17 |7        |60  |60      |96        |23        |881                   |7    |18   |53           |[]   |null  |null |
|4  |Darian   |132|132   |15 |46       |13  |13      |87        |432       |444                   |2    |23   |17           |[]   

In [16]:
for table in spark.catalog.listTables():
    print(table)

Table(name='armor', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)
Table(name='hero', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)
Table(name='weapon', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)


In [17]:
weapon_column = spark.sql(f'''SELECT weapon FROM hero 
                             WHERE weapon IS NOT NULL
                          ''').collect()

for row in weapon_column:
    print(row[0])

26


In [18]:

spark.sql(f'''SELECT * FROM weapon 
            WHERE weapon.name LIKE '%btarde%'
          ''').show()

spark.sql(f'''SELECT * FROM weapon 
            WHERE weapon.id == 26
          ''').show()

spark.sql(f'''SELECT * FROM hero 
            JOIN weapon on hero.weapon = weapon.id
          ''').show()

spark.sql(f'''SELECT weapon.* FROM hero 
            JOIN weapon on hero.weapon = weapon.id
          ''').show()

+---+---------+---+---------+----+---------+----------+--------------+------+
| id|     name|atk|atk_magik|mana|mana_cost|durability|max_durability|broken|
+---+---------+---+---------+----+---------+----------+--------------+------+
| 26|pe btarde|  9|        1|   2|        5|         3|            10| false|
+---+---------+---+---------+----+---------+----------+--------------+------+

+---+---------+---+---------+----+---------+----------+--------------+------+
| id|     name|atk|atk_magik|mana|mana_cost|durability|max_durability|broken|
+---+---------+---+---------+----+---------+----------+--------------+------+
| 26|pe btarde|  9|        1|   2|        5|         3|            10| false|
+---+---------+---+---------+----+---------+----------+--------------+------+

+---+------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+---+---------+---+---------+----+---------+----------+--------------+------+


In [19]:
spark.sql(f'''SELECT * 
            FROM hero 
            JOIN weapon ON hero.weapon = weapon.id
            WHERE hero.weapon IS NOT NULL
          ''').show()

+---+------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+---+---------+---+---------+----+---------+----------+--------------+------+
| id|  name| pv|max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor| id|     name|atk|atk_magik|mana|mana_cost|durability|max_durability|broken|
+---+------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+---+---------+---+---------+----+---------+----------+--------------+------+
| 26|Zander|175|   175|  1|       38|  77|      77|        61|       180|                   454|    2|   93|           41|   []|    26| null| 26|pe btarde|  9|        1|   2|        5|         3|            10| false|
+---+------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+

In [20]:
weapon_cols = [f"concat_ws('_', 'weapon') as weapon_{col}" for col in  spark.table("weapon").columns]
weapon_cols_str = ", ".join(weapon_cols)
weapon_cols_str



"concat_ws('_', 'weapon') as weapon_id, concat_ws('_', 'weapon') as weapon_name, concat_ws('_', 'weapon') as weapon_atk, concat_ws('_', 'weapon') as weapon_atk_magik, concat_ws('_', 'weapon') as weapon_mana, concat_ws('_', 'weapon') as weapon_mana_cost, concat_ws('_', 'weapon') as weapon_durability, concat_ws('_', 'weapon') as weapon_max_durability, concat_ws('_', 'weapon') as weapon_broken"

In [21]:

query = f"""
    SELECT hero.*, {weapon_cols_str}
    FROM hero
    JOIN weapon ON hero.weapon = weapon.id
    WHERE hero.weapon IS NOT NULL
"""

print(query)
spark.sql(query).show()


    SELECT hero.*, concat_ws('_', 'weapon') as weapon_id, concat_ws('_', 'weapon') as weapon_name, concat_ws('_', 'weapon') as weapon_atk, concat_ws('_', 'weapon') as weapon_atk_magik, concat_ws('_', 'weapon') as weapon_mana, concat_ws('_', 'weapon') as weapon_mana_cost, concat_ws('_', 'weapon') as weapon_durability, concat_ws('_', 'weapon') as weapon_max_durability, concat_ws('_', 'weapon') as weapon_broken
    FROM hero
    JOIN weapon ON hero.weapon = weapon.id
    WHERE hero.weapon IS NOT NULL

+---+------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+---------+-----------+----------+----------------+-----------+----------------+-----------------+---------------------+-------------+
| id|  name| pv|max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|weapon_id|weapon_name|weapon_atk|weapon_atk_magik|weapon_mana|weapon_mana_cost|

In [22]:
def armor_update_table(updated_df):
    # Mettre à jour la colonne "armor" avec les valeurs de la colonne "armor_updated"
    updated_df = updated_df.withColumn('armor', col('armor_updated'))

    # Supprimer la colonne "armor_updated"
    updated_df = updated_df.drop('armor_updated')
    
    return updated_df



# Créer une nouvelle colonne avec la valeur mise à jour conditionnellement
updated_df = df_hero.withColumn('armor_updated', when(col('id') == 18, 18).otherwise(col('armor')))


# Mettre à jour la colonne "armor" avec les valeurs de la colonne "armor_updated" et Supprimer la colonne "armor_updated" 
df_hero = armor_update_table(updated_df)

# Afficher le DataFrame mis à jour
df_hero.show()

+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
| id|     name| pv|max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|
+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
|  1|     Aria|195|   195| 13|       14|   5|       5|        54|       582|                   103|    9|   48|           96|   []|  null| null|
|  2|Balthazar|112|   112| 40|        6|  63|      63|        28|       237|                   587|    2|   83|           60|   []|  null| null|
|  3|Cassandra|160|   160| 17|        7|  60|      60|        96|        23|                   881|    7|   18|           53|   []|  null| null|
|  4|   Darian|132|   132| 15|       46|  13|      13|        87|       432|                   444|    2|   23|           17|   []

In [23]:
# Créer une nouvelle colonne avec la valeur mise à jour conditionnellement
updated_df = df_hero.withColumn('armor_updated', when((col('id') < 18) | (col('id') > 18), 14).otherwise(col('armor')))

# Mettre à jour la colonne "armor" avec les valeurs de la colonne "armor_updated" et Supprimer la colonne "armor_updated" 
df_hero = armor_update_table(updated_df)

# Afficher le DataFrame mis à jour
df_hero.show(n=50)

+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
| id|     name| pv|max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|
+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
|  1|     Aria|195|   195| 13|       14|   5|       5|        54|       582|                   103|    9|   48|           96|   []|  null|   14|
|  2|Balthazar|112|   112| 40|        6|  63|      63|        28|       237|                   587|    2|   83|           60|   []|  null|   14|
|  3|Cassandra|160|   160| 17|        7|  60|      60|        96|        23|                   881|    7|   18|           53|   []|  null|   14|
|  4|   Darian|132|   132| 15|       46|  13|      13|        87|       432|                   444|    2|   23|           17|   []

In [24]:
# Créer une nouvelle colonne avec la valeur mise à jour conditionnellement
updated_df = df_hero.withColumn('armor_updated', when(col('id').between(10, 15), 11).otherwise(col('armor')))


# Mettre à jour la colonne "armor" avec les valeurs de la colonne "armor_updated" et Supprimer la colonne "armor_updated" 
df_hero = armor_update_table(updated_df)

# Afficher le DataFrame mis à jour
df_hero.show()



+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
| id|     name| pv|max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|
+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
|  1|     Aria|195|   195| 13|       14|   5|       5|        54|       582|                   103|    9|   48|           96|   []|  null|   14|
|  2|Balthazar|112|   112| 40|        6|  63|      63|        28|       237|                   587|    2|   83|           60|   []|  null|   14|
|  3|Cassandra|160|   160| 17|        7|  60|      60|        96|        23|                   881|    7|   18|           53|   []|  null|   14|
|  4|   Darian|132|   132| 15|       46|  13|      13|        87|       432|                   444|    2|   23|           17|   []

In [25]:
# Créer une nouvelle colonne avec la valeur mise à jour conditionnellement
updated_df = df_hero.withColumn('armor_updated', when((col('armor') == 14) & (col('id') < 5), 7).otherwise(col('armor')))


# Mettre à jour la colonne "armor" avec les valeurs de la colonne "armor_updated" et Supprimer la colonne "armor_updated" 
df_hero = armor_update_table(updated_df)

# Afficher le DataFrame mis à jour
df_hero.show()

+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
| id|     name| pv|max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|
+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
|  1|     Aria|195|   195| 13|       14|   5|       5|        54|       582|                   103|    9|   48|           96|   []|  null|    7|
|  2|Balthazar|112|   112| 40|        6|  63|      63|        28|       237|                   587|    2|   83|           60|   []|  null|    7|
|  3|Cassandra|160|   160| 17|        7|  60|      60|        96|        23|                   881|    7|   18|           53|   []|  null|    7|
|  4|   Darian|132|   132| 15|       46|  13|      13|        87|       432|                   444|    2|   23|           17|   []

In [26]:
# Créer une nouvelle colonne avec la valeur mise à jour conditionnellement
updated_df = df_hero.withColumn('armor_updated', when((col('armor') == 18) , 'null' ).otherwise(col('armor')))


# Mettre à jour la colonne "armor" avec les valeurs de la colonne "armor_updated" et Supprimer la colonne "armor_updated" 
df_hero = armor_update_table(updated_df)

# Afficher le DataFrame mis à jour
df_hero.show()

+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
| id|     name| pv|max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|
+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
|  1|     Aria|195|   195| 13|       14|   5|       5|        54|       582|                   103|    9|   48|           96|   []|  null|    7|
|  2|Balthazar|112|   112| 40|        6|  63|      63|        28|       237|                   587|    2|   83|           60|   []|  null|    7|
|  3|Cassandra|160|   160| 17|        7|  60|      60|        96|        23|                   881|    7|   18|           53|   []|  null|    7|
|  4|   Darian|132|   132| 15|       46|  13|      13|        87|       432|                   444|    2|   23|           17|   []

In [28]:
spark.sql(f'''SELECT * 
            FROM hero  
          ''').show()
df_hero.write.mode('overwrite').saveAsTable('hero')
spark.sql(f'''SELECT * 
            FROM hero  
          ''').show()

+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
| id|     name| pv|max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|
+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
|  1|     Aria|195|   195| 13|       14|   5|       5|        54|       582|                   103|    9|   48|           96|   []|  null| null|
|  2|Balthazar|112|   112| 40|        6|  63|      63|        28|       237|                   587|    2|   83|           60|   []|  null| null|
|  3|Cassandra|160|   160| 17|        7|  60|      60|        96|        23|                   881|    7|   18|           53|   []|  null| null|
|  4|   Darian|132|   132| 15|       46|  13|      13|        87|       432|                   444|    2|   23|           17|   []

Py4JJavaError: An error occurred while calling o150.saveAsTable.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalog.validateTableLocation(SessionCatalog.scala:413)
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.run(createDataSourceTables.scala:176)
	